In [1]:
# Suppress noisy XLA/TensorFlow GPU logs
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import torch
from torch.utils.data import DataLoader
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    get_scheduler,
    DataCollatorForSeq2Seq
)
from torch.optim import AdamW
from datasets import load_dataset
from accelerate import Accelerator
from tqdm import tqdm
import numpy as np

# Accelerator for multi-GPU/TPU
accelerator = Accelerator()

# Load tokenizer & model 
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Load dataset
raw_data = load_dataset("json", data_files="/kaggle/input/travelmate/merged_augmented.jsonl")["train"]

# Train/Validation split
split_dataset = raw_data.train_test_split(test_size=0.1, seed=42)
train_data = split_dataset["train"]
val_data = split_dataset["test"]

# Tokenization
def tokenize(example):
    model_inputs = tokenizer(
        example["input"], truncation=True, padding="max_length", max_length=512
    )
    labels = tokenizer(
        example["target"], truncation=True, padding="max_length", max_length=128
    )
    # Convert labels to NumPy array to avoid slow tensor creation warning
    model_inputs["labels"] = np.array(labels["input_ids"], dtype=np.int64)
    return model_inputs

train_tokenized = train_data.map(tokenize, batched=True)
val_tokenized = val_data.map(tokenize, batched=True)

# Keep only tensors for training
train_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Dataloaders
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
train_loader = DataLoader(train_tokenized, batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_tokenized, batch_size=8, shuffle=False, collate_fn=data_collator)

# Optimizer + Scheduler
optimizer = AdamW(model.parameters(), lr=3e-4)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_loader) * 5,
)

# Prepare everything for accelerator
model, optimizer, train_loader, val_loader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_loader, val_loader, lr_scheduler
)

# Training Loop
for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"✅ Epoch {epoch+1} Train Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
    print(f"🔍 Epoch {epoch+1} Validation Loss: {val_loss / len(val_loader):.4f}")


E0000 00:00:1755000888.046736      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755000888.100644      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

🔍 Epoch 4 Validation Loss: 0.0083


Epoch 5 [Train]: 100%|██████████| 13491/13491 [46:54<00:00,  4.79it/s]


✅ Epoch 5 Train Loss: 0.0079


Epoch 5 [Val]: 100%|██████████| 1499/1499 [01:48<00:00, 13.87it/s]

🔍 Epoch 5 Validation Loss: 0.0077


In [ ]:
# Install evaluate if not already installed
!pip install evaluate sacrebleu rouge_score

import evaluate

# Save model & tokenizer
save_dir = "./t5_travelmate_model"
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(save_dir, save_function=accelerator.save)
tokenizer.save_pretrained(save_dir)
print(f"✅ Model saved to {save_dir}")

# Load ROUGE & BLEU metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")

# Evaluate on validation set
model.eval()
preds, refs = [], []

for batch in tqdm(val_loader, desc="Evaluating"):
    with torch.no_grad():
        outputs = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=128
        )
    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    decoded_refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

    preds.extend(decoded_preds)
    refs.extend([[ref] for ref in decoded_refs])  # BLEU expects list of lists

# Compute metrics
rouge_result = rouge.compute(predictions=preds, references=[r[0] for r in refs])
bleu_result = bleu.compute(predictions=preds, references=refs)

print(f"📏 ROUGE-1: {rouge_result['rouge1']:.4f}")
print(f"📏 ROUGE-L: {rouge_result['rougeL']:.4f}")
print(f"📏 BLEU: {bleu_result['score']:.2f}")

# Show a few predictions
for i in range(3):
    print(f"Input: {tokenizer.decode(val_loader.dataset[i]['input_ids'], skip_special_tokens=True)}")
    print(f"Prediction: {preds[i]}")
    print(f"Reference: {refs[i][0]}")
    print("-"*50)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d8f050ef0521fc3bb735dda8ab26ad75425ff23fb0b7d5d7adc638a2028f4181
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigfra

Evaluating: 100%|██████████| 1499/1499 [23:33<00:00,  1.06it/s]


📏 ROUGE-1: 0.9804
📏 ROUGE-L: 0.9793
📏 BLEU: 96.18
Input: USER: I am looking for information in Cambridge SYSTEM: What kind of information were you looking for? USER: I would like a 4 star luxury place to stay. It must include free wifi. But I'm cheap as all get out, so nothing too pricey. SYSTEM: I have a few that match, do you have a location preference? USER: The area doesn't matter as long as they have a room available for 3 people staying 4 nights. SYSTEM: That shouldn't be a problem. What day would you like to begin your stay? USER: from Thursday. SYSTEM: alexander bed and breakfast would be perfect for you. Shall i book it? USER: It'll be very good if you can include alexander bed and breakfast but need to be in a reasonable price.
Prediction: intents: book_hotel; slots: hotel-area=dontcare; hotel-bookday=thursday; hotel-bookpeople=3; hotel-bookstay=4; hotel-internet=yes; hotel-name=alexander bed and breakfast; hotel-pricerange=cheap; hotel-stars=4
Reference: intents: book_hotel;

In [7]:
import shutil
shutil.make_archive('t5_travelmate_model', 'zip', './t5_travelmate_model')

'/kaggle/working/t5_travelmate_model.zip'